In [1]:
import pandas as pd

In [2]:
api_logs = pd.read_csv("API_logs.csv", low_memory=False)

In [3]:
api_logs.shape

(5372177, 27)

In [4]:
report_logs = pd.read_csv("Report_newlogs.csv", low_memory=False)

In [5]:
report_logs.shape

(367182, 31)

In [6]:
cols = set(list(api_logs.columns) + list(report_logs.columns))

In [7]:
df = pd.DataFrame({
    "ColumnName" : [c for c in cols], 
    "API": [c in api_logs for c in cols], 
    "Report": [c in report_logs for c in cols]})

In [8]:
df[df.API & df.Report].ColumnName

0       ORGANIZATION_ID
1            REQUEST_ID
3            EVENT_TYPE
5                   URI
6             LOGIN_KEY
8           DB_CPU_TIME
9           SESSION_KEY
11    TIMESTAMP_DERIVED
12        DB_TOTAL_TIME
14             CPU_TIME
15       REQUEST_STATUS
18              USER_ID
22            DB_BLOCKS
24      USER_ID_DERIVED
27          ENTITY_NAME
28            CLIENT_IP
29             RUN_TIME
32       URI_ID_DERIVED
34            USER_TYPE
35            TIMESTAMP
Name: ColumnName, dtype: object

In [9]:
common_columns = ['TIMESTAMP_DERIVED', 'CLIENT_IP', 'ORGANIZATION_ID', 'USER_TYPE']

In [10]:
api_logs.dropna(subset=common_columns, inplace=True)

In [11]:
report_logs.dropna(subset=common_columns, inplace=True)

In [12]:
print(api_logs.shape, report_logs.shape)

(4450221, 27) (290584, 31)


In [13]:
api_report_logs = pd.merge(left=api_logs, right=report_logs, on=common_columns)

In [17]:
api_report_logs[['TIMESTAMP_DERIVED', 'REQUEST_SIZE', 'REPORT_ID', 'USER_ID_x', 'USER_ID_y']]

,TIMESTAMP_DERIVED,REQUEST_SIZE,REPORT_ID,USER_ID_x,USER_ID_y
0,2022-06-28T01:24:17.797Z,727.0,00O2R000004F15k,0050b000004KISt,0050b000004KD9z
1,2022-06-28T10:01:04.905Z,675.0,00O2R000003JCXa,0052R00000AQuu8,0050b000004KEAY
2,2022-06-28T10:01:45.551Z,651.0,00O0b000004kTaz,0050b000004KISt,0050b000004K3KF
3,2022-06-28T10:07:17.511Z,655.0,00O0b000004kTaz,0050b000004KISt,0050b000004K3KF
4,2022-06-28T10:16:07.089Z,662.0,00O2R0000043lHL,0050b000004KISt,0051P000003jYhs
...,...,...,...,...,...
62,2022-07-22T02:18:08.428Z,651.0,00O2R0000043l7v,0050b000004KISt,0050b000005Cdyy
63,2022-07-22T02:58:16.567Z,662.0,00O0b000006K6tL,0050b000004KISt,0050b000004KBmF
64,2022-07-22T02:09:56.614Z,660.0,00O2R000004EjYV,0050b000004KISt,0050b000004KK2Z
65,2022-07-22T02:11:23.242Z,643.0,00O0b000004kTaz,0050b000004KISt,0050b000004KEIe


In [18]:
sorted(api_report_logs)

['API_TYPE',
 'API_VERSION',
 'AVERAGE_ROW_SIZE',
 'CLIENT_IP',
 'CLIENT_NAME',
 'CPU_TIME_x',
 'CPU_TIME_y',
 'DB_BLOCKS_x',
 'DB_BLOCKS_y',
 'DB_CPU_TIME_x',
 'DB_CPU_TIME_y',
 'DB_TOTAL_TIME_x',
 'DB_TOTAL_TIME_y',
 'DISPLAY_TYPE',
 'ENTITY_NAME_x',
 'ENTITY_NAME_y',
 'EVENT_TYPE_x',
 'EVENT_TYPE_y',
 'LOGIN_KEY_x',
 'LOGIN_KEY_y',
 'METHOD_NAME',
 'NUMBER_BUCKETS',
 'NUMBER_COLUMNS',
 'NUMBER_EXCEPTION_FILTERS',
 'ORGANIZATION_ID',
 'ORIGIN',
 'RENDERING_TYPE',
 'REPORT_ID',
 'REQUEST_ID_x',
 'REQUEST_ID_y',
 'REQUEST_SIZE',
 'REQUEST_STATUS_x',
 'REQUEST_STATUS_y',
 'RESPONSE_SIZE',
 'ROWS_PROCESSED',
 'ROW_COUNT',
 'RUN_TIME_x',
 'RUN_TIME_y',
 'ReportId',
 'SESSION_KEY_x',
 'SESSION_KEY_y',
 'SORT',
 'TIMESTAMP_DERIVED',
 'TIMESTAMP_x',
 'TIMESTAMP_y',
 'URI_ID_DERIVED_x',
 'URI_ID_DERIVED_y',
 'URI_x',
 'URI_y',
 'USER_ID_DERIVED_x',
 'USER_ID_DERIVED_y',
 'USER_ID_x',
 'USER_ID_y',
 'USER_TYPE']

In [20]:
api_report_logs.to_csv("api_report_join.csv", index=False)

In [34]:
probl_reports_splunk = pd.read_csv("actives_and_problematics_regarding_time_last_30_days_from-27-07-2022.csv")

In [35]:
set(api_report_logs.REPORT_ID).intersection(set(probl_reports_splunk.ReportId.unique()))

{'00O0b000004AoOO', '00O0b000004kTaz'}

In [36]:
probl_reports_splunk[probl_reports_splunk.ReportId=='00O0b000004AoOO'].Name

13    Chat SLA - Wait Time
Name: Name, dtype: object

In [37]:
probl_reports_splunk[probl_reports_splunk.ReportId=='00O0b000004kTaz'].Name

18    My Cases and Tasks
Name: Name, dtype: object